In [163]:
from dateutil.relativedelta import relativedelta
from argon2 import PasswordHasher
from datetime import datetime
from faker import Faker
import pandas as pd
import random as rd
import re

SEED = 29
Faker.seed(SEED)
rd.seed(SEED)

ph = PasswordHasher()
fk = Faker('pt-BR')
QNT = 10

In [164]:
# FIXME: make the amount of addresses to be passed as parameter

def generate_addresses():
    complements = ['Casa 2', 'Apto. 162', 'Casa B', 'Galpão A', None, None, None, None, None, None]
    raw_addresses = []

    for ad in [fk.address() for _ in range(2 * QNT - 1)]:
        split_ad = ad.split('\n')
        street_address = split_ad[0].split(',')
        street = street_address[0]
        number = int(street_address[1]) if len(street_address) > 1 else 0
        temp, state = split_ad[2].split('/')
        postal_code, city = temp.split(' ', maxsplit=1)
        postal_code = postal_code.replace('-', '')
        complement = rd.choice(complements)

        raw_addresses.append(('Brasil', state, city, street, number, postal_code, complement))

    return raw_addresses

In [165]:
def id_generator(df: pd.DataFrame):
    next_id = 0
    while True:
        next_id = (next_id + 1) % (df.shape[0] + 1)
        if next_id == 0: continue
        yield next_id

In [166]:
addresses = pd.DataFrame(
    data=generate_addresses(),
    columns=['country', 'state', 'city', 'street', 'number', 'postal_code', 'complement']
)

seq_address_id = id_generator(addresses)

display(addresses)

,country,state,city,street,number,postal_code,complement
0,Brasil,MT,Monteiro,Trevo de da Paz,9,66001735,None
1,Brasil,SE,Jesus,Estrada de Machado,73,98386448,Apto. 162
2,Brasil,RS,Leão,Lagoa Sá,89,52425186,None
3,Brasil,AL,Silveira,Vale Leonardo Duarte,14,66826240,None
4,Brasil,SC,Alves,Vereda Moreira,0,08022753,None
5,Brasil,AM,Pinto,Alameda José Correia,3,63338353,None
6,Brasil,RO,Monteiro do Sul,Viela Leão,6,92461606,Apto. 162
7,Brasil,PE,Cavalcanti de Ramos,Distrito Cavalcante,89,53707393,None
8,Brasil,RN,Sampaio,Área Porto,90,40143531,None
9,Brasil,MA,Gonçalves dos Dourados,Colônia da Cunha,15,33634485,None


In [167]:
gen_date = lambda: fk.date_between(datetime.now() - relativedelta(years=50), datetime.now() - relativedelta(years=12))
gen_password = lambda size: fk.password(length=size)

users = pd.DataFrame({
    'cpf': [fk.cpf() for _ in range(QNT)],
    'profile_pic': [None] * QNT,
    'email': [fk.email() for _ in range(QNT)],
    'username': [fk.name() for _ in range(QNT)],
    'phone': [fk.cellphone_number() for _ in range(QNT)],
    'birthday': [gen_date() for _ in range(QNT)],
    'is_active': [True] * QNT,
    'raw_password': [gen_password(8) for _ in range(QNT)],
    'address_id': [next(seq_address_id) for _ in range(QNT)],
})

users['password'] = users['raw_password'].apply(ph.hash)
users['phone'] = users['phone'].apply(lambda x: re.sub(r'[^0-9]', '', x))
users.to_csv('csv/test_logins.csv', columns=['email', 'raw_password'])
users.drop(columns=['raw_password'], inplace=True)

display(users)

,cpf,profile_pic,email,username,phone,birthday,is_active,address_id,password
0,189.267.053-40,None,asafemartins@example.com,Ana Liz Aragão,5585928808034,2002-12-31,True,1,"$argon2id$v=19$m=65536,t=3,p=4$pU6x5bAF2tGmVJn..."
1,593.206.147-25,None,ana-clara79@example.com,Ágatha Dias,5588909349576,1991-01-18,True,2,"$argon2id$v=19$m=65536,t=3,p=4$Vmp4Ta/BjBEtSjt..."
2,392.810.756-95,None,sophiasales@example.org,Dr. Arthur Gabriel Camargo,5518940068655,1994-10-16,True,3,"$argon2id$v=19$m=65536,t=3,p=4$hkCr/oXZIlef7mR..."
3,794.256.801-67,None,ferreiraraquel@example.org,Dom Costa,5556946307849,2007-05-30,True,4,"$argon2id$v=19$m=65536,t=3,p=4$mOlZ9M0ejWWviy5..."
4,820.651.437-62,None,davi-lucca02@example.net,Rafael Cardoso,5560975892589,1976-10-03,True,5,"$argon2id$v=19$m=65536,t=3,p=4$zHd7VnblwsOtirO..."
5,253.067.984-38,None,maria-florda-cruz@example.net,Maria Isis Peixoto,5517929449515,1979-01-27,True,6,"$argon2id$v=19$m=65536,t=3,p=4$yu3fbLKyZ32VJqx..."
6,829.037.516-68,None,bryanvasconcelos@example.org,Laís Albuquerque,5594906012502,2009-05-11,True,7,"$argon2id$v=19$m=65536,t=3,p=4$3X+ol1eFi+V+HB6..."
7,245.369.718-82,None,pedro30@example.com,Thomas Gonçalves,5571934478274,1985-03-18,True,8,"$argon2id$v=19$m=65536,t=3,p=4$LO5W5XlgyJ9QKL/..."
8,586.097.412-49,None,portolaura@example.com,Dra. Eloah Fogaça,5519989153459,1985-12-13,True,9,"$argon2id$v=19$m=65536,t=3,p=4$SOjWfizhWsLSbQ4..."
9,784.026.591-01,None,nunesdavi@example.com,Marcela Vargas,55056935921444,1992-04-15,True,10,"$argon2id$v=19$m=65536,t=3,p=4$+JfH/pHiH/Qq5Lg..."


In [168]:
stores = pd.DataFrame({
    'cnpj': [fk.cnpj() for _ in range(QNT)],
    'name': [fk.company() for _ in range(QNT)],
    'picture_url': [None] * QNT,
    'email': [fk.company_email() for _ in range(QNT)],
    'is_active': [True] * QNT,
    'raw_password': [gen_password(8) for _ in range(QNT)],
    'address_id': [next(seq_address_id) for _ in range(QNT)]
})

stores['password'] = stores['raw_password'].apply(ph.hash)
stores.to_csv('csv/test_store_logins.csv', columns=['email', 'raw_password'])
stores.drop(columns=['raw_password'], inplace=True)

seq_store_id = id_generator(stores)

display(stores)

,cnpj,name,picture_url,email,is_active,address_id,password
0,98.720.645/0001-01,Guerra - EI,None,renanleao@nascimento.br,True,11,"$argon2id$v=19$m=65536,t=3,p=4$BB+ua+0kqK/Qvvk..."
1,80.423.567/0001-00,Rocha Lima - EI,None,gazevedo@aparecida.br,True,12,"$argon2id$v=19$m=65536,t=3,p=4$eLKwXcL7NjLfYLS..."
2,58.403.967/0001-02,Vasconcelos - ME,None,marcela49@pinto.com,True,13,"$argon2id$v=19$m=65536,t=3,p=4$QQ+1dc0KyEFeD3m..."
3,84.576.219/0001-23,da Conceição e Filhos,None,wnovais@pereira.com,True,14,"$argon2id$v=19$m=65536,t=3,p=4$wxKdNgfT70pFrOx..."
4,64.075.293/0001-59,Silveira,None,da-rochathomas@da.com,True,15,"$argon2id$v=19$m=65536,t=3,p=4$oS/QMQTlRLyVJQR..."
5,26.019.853/0001-69,Nunes Ltda.,None,melissa07@santos.com,True,16,"$argon2id$v=19$m=65536,t=3,p=4$O5RpcaTuGuNaVPC..."
6,27.136.580/0001-03,Costa,None,joseteixeira@andrade.net,True,17,"$argon2id$v=19$m=65536,t=3,p=4$wDalNoZra5jLVO6..."
7,70.952.438/0001-55,da Rosa,None,hcamara@albuquerque.org,True,18,"$argon2id$v=19$m=65536,t=3,p=4$s7VG++dIBFe+9zO..."
8,38.764.501/0001-17,Aparecida e Filhos,None,vgarcia@da.br,True,19,"$argon2id$v=19$m=65536,t=3,p=4$1JRb1dF7vm2f4Rd..."
9,73.082.961/0001-66,Vieira da Paz - EI,None,diogorocha@oliveira.com,True,1,"$argon2id$v=19$m=65536,t=3,p=4$10GhgDbpPlUBRoV..."


In [169]:
store_phones = pd.DataFrame({
    'phone': [fk.cellphone_number() for _ in range(QNT + QNT // 5)],
    'store_id': [next(seq_store_id) for _ in range(QNT + QNT // 5)]
})

store_phones['phone'] = store_phones['phone'].apply(lambda x: re.sub(r'[^0-9+]', '', x))

display(store_phones)

,phone,store_id
0,+5564977593885,1
1,+5536907836542,2
2,+55005978577911,3
3,+5548963368052,4
4,+5551947663909,5
5,+5568972248822,6
6,+5532948585088,7
7,+5568969396043,8
8,+5551998912977,9
9,+5512925722675,10


In [170]:
products = pd.DataFrame({
    'name': [f'{fk.word().capitalize()} {fk.word()}' for _ in range(QNT * 10)],
    'price': [round(rd.uniform(10, 100), 2) for _ in range(QNT * 10)],
    'attributes': [None] * QNT * 10,
    'store_id': [next(seq_store_id) for _ in range(QNT * 10)]
})

display(products)

,name,price,attributes,store_id
0,Aliquam expedita,39.64,None,3
1,Iste error,64.70,None,4
2,Ullam tempora,47.77,None,5
3,Iure repellendus,75.74,None,6
4,Excepturi consectetur,50.25,None,7
...,...,...,...,...
95,Est laudantium,49.14,None,8
96,Voluptatibus ea,44.05,None,9
97,Accusamus necessitatibus,55.00,None,10
98,Quibusdam molestiae,90.55,None,1
